In [9]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import InceptionV3
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.models import Model
from keras.optimizers import Adam
import cv2 as cv
import os
from matplotlib import pyplot as plt
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [2]:
train_dir = 'train_set_ex'
validation_dir = 'validation_set_ex'
test_dir = 'test_set_ex'

In [3]:
# Create ImageDataGenerators for training, validation, and testing
batch_size = 32
target_size = (256, 256)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(rescale=1. / 255)

test_datagen = ImageDataGenerator(rescale=1. / 255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_dataset = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=1,  # Set batch_size to 1 for testing
    class_mode='categorical',
    shuffle=False
)

Found 3267 images belonging to 12 classes.
Found 1267 images belonging to 12 classes.
Found 564 images belonging to 12 classes.


In [4]:
train_dataset.class_indices

{'Bakso': 0,
 'Caesar Salad': 1,
 'Kerang Tiram': 2,
 'Nasi': 3,
 'Nasi Goreng': 4,
 'Sate Ayam': 5,
 'Sayap Ayam Goreng': 6,
 'Siomay': 7,
 'Spaghetti': 8,
 'Steak': 9,
 'Telur Balado': 10,
 'Yoghurt': 11}

In [5]:
# # Set the number of batches to visualize
# num_batches_to_visualize = 3

# # Iterate over batches and show images
# for batch_num in range(3):
#     # Get a batch from the training dataset
#     images, labels = train_dataset.next()

#     # Display the images in a grid
#     plt.figure(figsize=(12, 12))
#     for i in range(batch_size):
#         plt.subplot(2, 4, i + 1)
#         plt.imshow(images[i])
        
#         # Assuming you have a list of class names
#         classes = train_dataset.class_indices
#         class_index = np.argmax(labels[i])
#         class_name = [k for k, v in classes.items() if v == class_index][0]
        
#         plt.title(class_name)
#         plt.axis('off')

#     plt.show()

In [10]:
#Model Architecture

model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(256, 256, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(train_dataset.num_classes, activation='softmax')
])


model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_dataset.num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

class CustomEarlyStopping(Callback):
    def on_epoch_end(self, epoch, logs=None):
        accuracy = logs.get('accuracy')
        val_accuracy = logs.get('val_accuracy')
        if accuracy is not None and val_accuracy is not None:
            if accuracy > 0.85 and val_accuracy > 0.85:
                print(f"\nStopping training as both training accuracy and validation accuracy have exceeded 0.85: \nTraining Accuracy: {accuracy}, Validation Accuracy: {val_accuracy}")
                self.model.stop_training = True

custom_early_stopping = CustomEarlyStopping()

history = model.fit(
    train_dataset,
    epochs=200,
    validation_data=validation_generator,
    callbacks=[custom_early_stopping]
)

# Evaluate the model
evaluation = model.evaluate(test_dataset)
print(f"Test Loss: {evaluation[0]}")
print(f"Test Accuracy: {evaluation[1]}")

Epoch 1/200
103/103 [==============================] - 36s 345ms/step - loss: 2.2103 - accuracy: 0.2063 - val_loss: 2.0253 - val_accuracy: 0.3204
Epoch 2/200
103/103 [==============================] - 34s 328ms/step - loss: 1.9350 - accuracy: 0.3159 - val_loss: 1.8164 - val_accuracy: 0.3915
Epoch 3/200
103/103 [==============================] - 34s 334ms/step - loss: 1.8263 - accuracy: 0.3612 - val_loss: 1.6970 - val_accuracy: 0.4309
Epoch 4/200
103/103 [==============================] - 33s 322ms/step - loss: 1.7498 - accuracy: 0.4034 - val_loss: 1.6600 - val_accuracy: 0.4522
Epoch 5/200
103/103 [==============================] - 36s 350ms/step - loss: 1.6930 - accuracy: 0.4249 - val_loss: 1.6290 - val_accuracy: 0.4799
Epoch 6/200
103/103 [==============================] - 33s 323ms/step - loss: 1.6405 - accuracy: 0.4539 - val_loss: 1.5929 - val_accuracy: 0.4728
Epoch 7/200
103/103 [==============================] - 33s 321ms/step - loss: 1.5766 - accuracy: 0.4607 - val_loss: 1.5027 -

In [7]:
from sklearn.metrics import classification_report
import numpy as np

predictions = model.predict(test_dataset)
predicted_classes = np.argmax(predictions, axis=1)

true_classes = test_dataset.classes
class_labels = list(test_dataset.class_indices.keys())

report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

564/564 [==============================] - 2s 3ms/step
                   precision    recall  f1-score   support

            Bakso       1.00      0.22      0.36        50
     Caesar Salad       0.62      0.70      0.66        50
     Kerang Tiram       0.51      0.70      0.59        50
             Nasi       0.91      0.81      0.85        36
      Nasi Goreng       0.65      0.63      0.64        49
        Sate Ayam       0.91      0.44      0.59        68
Sayap Ayam Goreng       0.52      0.64      0.57        50
           Siomay       0.70      0.78      0.74        51
        Spaghetti       0.52      0.80      0.63        50
            Steak       0.51      0.66      0.57        50
     Telur Balado       0.60      0.30      0.40        10
          Yoghurt       0.71      0.70      0.71        50

         accuracy                           0.63       564
        macro avg       0.68      0.62      0.61       564
     weighted avg       0.69      0.63      0.62       564